In [6]:
import nibabel as nib
from glob import glob
import os
from tqdm import tqdm
import numpy as np
import pandas as pd

In [7]:
def load_lidcset(idx, base_dir = "../data/NII_normwinall"): #get img, mask
    img_lst = sorted(glob(os.path.join(base_dir, "*img.nii.gz")))
    mask_lst = sorted(glob(img_lst[idx].replace("img", "mask*")))
    img = nib.load(img_lst[idx]).get_fdata()
    nod = np.array(nib.load(mask_lst[0]).get_fdata())
    
    return img, nod

def normalize255(arr):
    return np.array(((arr - np.min(arr)) / (np.max(arr) - np.min(arr)))*255).astype(np.uint8)

def subtract_nodule(img, nod):
    return np.where(nod!=0, np.min(img), img)

def save_nod(nod_img, save_dir = "../data/NII_nod", num=0): #num for name
    nib.save(nod_img, f'{save_dir}/nod_{num}')

def make_drr(img):
    return np.mean(img.T, axis=2)


## Save all nodule

In [ ]:
img_list = sorted(glob("../data/NII_normwinall/*img*"))
img_len = len(img_list)

for img_idx in tqdm(range(img_len)):
    mask_list = sorted(glob(img_list[img_idx].replace("img", "mask*")))
    mask_len = len(mask_list)
    
    img=nib.load(img_list[img_idx]).get_fdata()
    img = np.uint8(img)

    for mask_idx in range(mask_len):
        nod = np.array(nib.load(mask_list[mask_idx]).get_fdata())
        nod = np.uint8(nod)
        nod_pos = np.where(nod > 35) # smoothing 35
        
        nod_only = nod[min(nod_pos[0]):max(nod_pos[0]), min(nod_pos[1]):max(nod_pos[1]), min(nod_pos[2]):max(nod_pos[2])]
        nod_nib = nib.Nifti1Image(nod_only, affine=np.eye(4))
        
        filename = mask_list[mask_idx].split('/')[-1].split('_')[0]
        nib.save(nod_nib, f'../data/NII_nod_smoothing/nod_{filename}_{mask_idx+1}.nii.gz')

## Save all nodule with Smoothing

In [ ]:
with open('../nod_smoothing.csv', 'r') as f:
    df = pd.read_csv(f)
df_len = len(df)

nod_height = []
nod_width = []
nod_depth = []
drop_img = []

In [124]:
for idx in tqdm(range(df_len)):
    
    if (df.height.iloc[idx] == 0) & (df.width.iloc[idx] == 0) & (df.depth.iloc[idx]==0):
        nod_height.append(0)
        nod_width.append(0)
        nod_depth.append(0)
        drop_img.append(idx)
        continue
    
    img_path = os.path.join('..', df.img_path.iloc[idx])
    mask_path =  os.path.join('..', df.nod_path.iloc[idx])
    
    if os.path.exists(img_path) and os.path.exists(mask_path) == False:
        nod_height.append(0)
        nod_width.append(0)
        nod_depth.append(0)
        drop_img.append(idx)
        continue
    
    img = nib.load(img_path).get_fdata()
    img = np.uint8(img)

    nod = np.array(nib.load(mask_path).get_fdata())
    nod = np.uint8(nod)
    nod_pos = np.where(nod > 35)
    
    if len(nod_pos[0]) == 0:
        nod_height.append(0)
        nod_width.append(0)
        nod_depth.append(0)        
        drop_img.append(idx)
        continue
    
    nod_only = nod[min(nod_pos[0]):max(nod_pos[0]), min(nod_pos[1]):max(nod_pos[1]), min(nod_pos[2]):max(nod_pos[2])]
    nod_shape = nod_only.shape
    
    nod_height.append(nod_shape[2])
    nod_width.append(nod_shape[1])
    nod_depth.append(nod_shape[0])
    
    nod_nib = nib.Nifti1Image(nod_only, affine=np.eye(4))

    filename = mask_path.split('/')[-1].split('_')[0]
    mask_num = mask_path.split('/')[-1].split('_')[-1].split('.')[0]
    nib.save(nod_nib, f'../data/NII_nod_smooth/nod_{filename}_{mask_num}.nii.gz')


100%|██████████| 2925/2925 [45:33<00:00,  1.07it/s]  


In [126]:
# modify and save nodule.csv
del df['height']
del df['width']
del df['depth']

df['height'] = nod_height
df['width'] = nod_width 
df['depth'] = nod_depth

df.to_csv('../nod_smoothing_35.csv', index=False)

### Save nodule > 10mm (20mm와 동일)

In [32]:
with open('../nod_smoothing_10_10mm.csv', 'r') as f:
    df = pd.read_csv(f)

In [ ]:
#save nodule > 10mm, 20mm

df_len = len(df)
img_num = 1

for idx in tqdm(range(df_len)):
    img_path = os.path.join('..', df.img_path.iloc[idx])
    mask_path =  os.path.join('..', df.nod_path.iloc[idx])
    
    img = nib.load(img_path).get_fdata()
    img = np.uint8(img)

    nod = np.array(nib.load(mask_path).get_fdata())
    nod = np.uint8(nod)
    nod_pos = np.where(nod >= 10) # smoothing 10

    nod_only = nod[min(nod_pos[0]):max(nod_pos[0]), min(nod_pos[1]):max(nod_pos[1]), min(nod_pos[2]):max(nod_pos[2])]
    nod_nib = nib.Nifti1Image(nod_only, affine=np.eye(4))

    filename = mask_path.split('/')[-1].split('_')[0]
    mask_num = mask_path.split('/')[-1].split('_')[-1].split('.')[0]
    
    if os.path.exists(f'../data/NII_nod_smooth10_10/nod_{filename}_{mask_num}.nii.gz') == False:
        nib.save(nod_nib, f'../data/NII_nod_smooth10_10/nod_{filename}_{mask_num}.nii.gz')
        img_num = 1
    else:
        nib.save(nod_nib, f'../data/NII_nod_smooth10_10/nod_{filename}_{img_num}_{mask_num}.nii.gz')
        img_num += 1
